暫定スコア19.774381	最終スコア14.284118

In [1]:
import pandas as pd
from ydata_profiling import ProfileReport
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import numpy as np
import seaborn as sns
%matplotlib inline


In [2]:
test_df = pd.read_csv("test.csv")
train_df = pd.read_csv("train.csv")
weather = pd.read_csv("weather.csv")
sample = pd.read_csv("sample_submission.csv")

In [3]:
train_df

,kind,date,amount,mode_price,area
0,だいこん,20051107,201445,735.0,千葉
1,だいこん,20051108,189660,840.0,千葉_各地_青森
2,だいこん,20051110,218166,735.0,千葉_各地_青森
3,だいこん,20051111,182624,682.5,千葉_青森
4,だいこん,20051112,220691,682.5,千葉_青森
...,...,...,...,...,...
97777,しめじ,20170526,17231,48.5,新潟_長野
97778,しめじ,20170527,17105,48.5,新潟_長野
97779,しめじ,20170529,12406,48.5,新潟_長野
97780,しめじ,20170530,12320,48.5,新潟_長野


In [4]:
sample

,kind,date,mode_price
0,だいこん,20220502,0
1,だいこん,20220506,0
2,だいこん,20220507,0
3,だいこん,20220509,0
4,だいこん,20220510,0
...,...,...,...
315,ミニトマト,20220526,0
316,ミニトマト,20220527,0
317,ミニトマト,20220528,0
318,ミニトマト,20220530,0


In [5]:
#profile = ProfileReport(train_df, title="Profiling Report")
#profile.to_notebook_iframe()
#profile.to_file("your_report.html")
# amount削除
train_df = train_df.drop(columns=['amount'])


In [6]:
# カテゴリ変数をcategory型に変換 
# 日付データを年数、月、日に分割


def split_location(value):
    parts = value.split("_")
    first = parts[0] if len(parts) > 0 else None
    second = None
    kakuchi = 0

    # Detect "各地"
    if "各地" in parts:
        kakuchi = 1
        parts_clean = [p for p in parts if p != "各地"]
        if len(parts_clean) >= 2:
            second = parts_clean[1]
        elif len(parts_clean) == 1:
            second = None
    else:
        if len(parts) >= 2:
            second = parts[1]

    return pd.Series([first, second, kakuchi])# areaをリスト型に
def Data_cre(df):


    df["date"] = df["date"].astype("str")
    df["date"] = pd.to_datetime(df["date"])
    
    df["year"] = df["date"].dt.year
    df["month"] = df["date"].dt.month
    df["day"] = df["date"].dt.day
    #df["date_only"] = df["date"].dt.date  # datetime.date 型
    df = df.drop(columns=['date'])

    df['kind'] = df['kind'].astype('category')
    # areaを分割する
    # df[["area_1", "area_2", "area_flag"]] = df["area"].apply(split_location)
    df['area'] = df['area'].astype('category')
    # df['area_1'] = df['area_1'].astype('category')
    # df['area_2'] = df['area_2'].astype('category')


    
    return df

train_df = Data_cre(train_df)
test_df = Data_cre(test_df) 



In [7]:
train_df["area"]

0              千葉
1        千葉_各地_青森
2        千葉_各地_青森
3           千葉_青森
4           千葉_青森
           ...   
97777       新潟_長野
97778       新潟_長野
97779       新潟_長野
97780       新潟_長野
97781       新潟_長野
Name: area, Length: 97782, dtype: category
Categories (266, object): ['アメリカ', 'アメリカ_カナダ', 'アメリカ_中国', 'アメリカ_北海道', ..., '香川_高知', '高知', '高知_鹿児島', '鹿児島']

In [8]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97782 entries, 0 to 97781
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   kind        97782 non-null  category
 1   mode_price  97782 non-null  float64 
 2   area        97782 non-null  category
 3   year        97782 non-null  int32   
 4   month       97782 non-null  int32   
 5   day         97782 non-null  int32   
dtypes: category(2), float64(1), int32(3)
memory usage: 2.2 MB


In [9]:
X = train_df.drop(['mode_price'], axis=1)
y = train_df['mode_price']
# 学習データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=0)

In [10]:
### chatgpt

models = {}  # kind → 学習済みモデルを保存する辞書

for kd, group in train_df.groupby("kind",observed=False):

    print(f"--- {kd} モデルを学習中 ---")

    X = group.drop(columns=["mode_price"],axis=1)
    y = group["mode_price"]

    # ← ここで kind ごとに train/test 分割
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_test, y_test)

    params = {
        'objective': 'regression',
        'num_leaves': 5,
        'seed': 0,
        'verbose': -1,
    }

    model = lgb.train(params,
                      lgb_train,
                      num_boost_round=1000,
                      valid_sets=[lgb_train, lgb_eval],
                      valid_names=['train', 'eval'],
                      callbacks=[lgb.early_stopping(stopping_rounds=10, verbose=True),lgb.log_evaluation(10)])

    models[kd] = model  # 保存

--- いんげん モデルを学習中 ---
Training until validation scores don't improve for 10 rounds
[10]	train's l2: 271010	eval's l2: 273808
[20]	train's l2: 252029	eval's l2: 257607
[30]	train's l2: 239436	eval's l2: 247589
[40]	train's l2: 231198	eval's l2: 245449
[50]	train's l2: 223813	eval's l2: 241730
[60]	train's l2: 217231	eval's l2: 235442
[70]	train's l2: 211243	eval's l2: 231632
[80]	train's l2: 207624	eval's l2: 230095
[90]	train's l2: 203170	eval's l2: 227480
[100]	train's l2: 199710	eval's l2: 225091
[110]	train's l2: 195432	eval's l2: 222403
[120]	train's l2: 193421	eval's l2: 220549
[130]	train's l2: 188767	eval's l2: 218120
[140]	train's l2: 185161	eval's l2: 214608
[150]	train's l2: 180415	eval's l2: 211302
[160]	train's l2: 177296	eval's l2: 208837
[170]	train's l2: 175543	eval's l2: 208430
[180]	train's l2: 173456	eval's l2: 207601
[190]	train's l2: 171166	eval's l2: 206782
[200]	train's l2: 168961	eval's l2: 205367
[210]	train's l2: 167194	eval's l2: 204296
[220]	train's l2: 165139

In [11]:
### chatgpt

pred_list = []

for kd, group in test_df.groupby("kind"):
    model = models[kd]       # 野菜ごとのモデルを取り出す
    X_test = group.copy()
    pred = model.predict(X_test)

    X_test["mode_price"] = pred
    pred_list.append(X_test)

result = pd.concat(pred_list).sort_index()

result["date"] = (
    result["year"].astype(str)
    + result["month"].astype(str).str.zfill(2)
    + result["day"].astype(str).str.zfill(2)
)

result["date"] = result["date"].astype(int)

submit = result[["kind", "date", "mode_price"]]
submit.to_csv("submit.csv", index=False)

C:\Users\ND013\AppData\Local\Temp\ipykernel_12008\4231220713.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for kd, group in test_df.groupby("kind"):
